In [1]:
from utils.PandasToolsFunction import *
from utils.DataPreprocessingTools import *
from utils.loss_functions import mean_absolute_percentage_error

from torch.nn.functional import mse_loss

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import json

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
# Define the linear regression model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        self.hidden_layers = torch.nn.ModuleList([torch.nn.Linear(hidden_dim, hidden_dim) for _ in range(n_layers - 1)])

        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.activation = torch.nn.ReLU()

    def forward(self, x): #week, day_numeric, day_type):
        """
        input: week, day_numeric, day_type (normalized)
        output: validations_nb
        """
        x = x.float()

        # normalize
        x[:,0] /= 52.
        x[:,1] /= 7.

        #print("x:", x)

        x = self.activation(self.input_layer(x.float()))
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        return self.output_layer(x)

In [3]:
class NN_Model():
    def __init__(self, input_dim, hidden_dim, n_layers, train_data, test_size):
        self.train_data = train_data
        self.stations = train_data['station'].unique()
        self.datasets = {}
        self.nns = {}
        self.features_col = ['week', 'day_numeric', 'day_type']

        """
        self.scaler = MinMaxScaler()
        mask = (train_data['station'] == train_data['station'].unique()[0])
        features = train_data[mask][self.features_col]
        _ = self.scaler.fit_transform(features)
        """

        self.loss = mse_loss
        #self.loss = mean_absolute_percentage_error
        #self.loss = torch.nn.SmoothL1Loss

        for station in self.stations :
            # create datasets
            station_indices = train_data.index[train_data['station'] == station]
            features = train_data.loc[station_indices, self.features_col]
            features = features.values
            y = train_data.loc[station_indices, 'y']
            y = np.array(y)
            #X_normalized = self.scaler.transform(features)
            self.datasets[station] = train_test_split(features, y, test_size=test_size, random_state= 42, shuffle=True)

            # creates nns
            self.nns[station] = NeuralNetwork(input_dim, hidden_dim, n_layers, 1)
    
    def train(self, nb_epochs, learning_rate):
        for station in tqdm(self.stations):

            optimizer = optim.Adam(self.nns[station].parameters(), lr=learning_rate)

            X_train, X_test, y_train, y_test = self.datasets[station]

            for epoch in tqdm(range(nb_epochs), leave=False):
                # Forward pass
                X_train, X_test = torch.tensor(X_train, device=device), torch.tensor(X_test, device=device)
                y_train, y_test = torch.tensor(y_train, device=device), torch.tensor(y_test, device=device)
                y_pred = self.nns[station](X_train)
                loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            test_pred = self.nns[station](X_test)
            print('Score on test set:', mean_absolute_percentage_error(y_test, test_pred).item())
            """
            print("y pred:", test_pred[0].item(), " y:", y_test[0].item())
            print('y features:', X_test[0])
            print("y pred:", test_pred[1].item(), " y:", y_test[1].item())
            print("y pred:", test_pred[2].item(), " y:", y_test[2].item())
            print("y pred:", test_pred[3].item(), " y:", y_test[3].item())
            print("y pred:", test_pred[4].item(), " y:", y_test[4].item())
            print('y features:', X_test[4])
            """
    
    def predict(self, station, x): #station, week, day_numeric, day_type):
        return self.nn[station](x)


In [4]:
input_dim = 3
n_layers = 2
hidden_dim = 512

train_data = pd.read_csv('data/train_data_day_typed_cov_replaced_all_features.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = date_filter(train_data, start_date='2019-12-31', end_date='2022-12-31')

test_size = 0.01

nn_model = NN_Model(input_dim, hidden_dim, n_layers, train_data, test_size)

In [7]:
nn_model.train(nb_epochs=50, learning_rate=0.001)

  0%|          | 0/439 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipykernel_67890/3640112851.py:45: UserWarning: Using a target size (torch.Size([770])) that is different to the input size (torch.Size([770, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)
/tmp/ipykernel_67890/3640112851.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test = torch.tensor(X_train, device=device), torch.tensor(X_test, device=device)
/tmp/ipykernel_67890/3640112851.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train, y_test = torch.tensor(y_train, device=device), torch.tensor(y_test, 

Score on test set: 1.0572460889816284


  0%|          | 0/50 [00:00<?, ?it/s]

...

/tmp/ipykernel_67890/3640112851.py:45: UserWarning: Using a target size (torch.Size([138])) that is different to the input size (torch.Size([138, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)


Score on test set: 0.4573074281215668


  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipykernel_67890/3640112851.py:45: UserWarning: Using a target size (torch.Size([272])) that is different to the input size (torch.Size([272, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)


Score on test set: 0.3899626135826111


In [8]:
test_data = pd.read_csv('data/test_data_day_typed_cov_replaced_all_features.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data

,index,date,week,day_name,day_numeric,station,day_type
0,2023-01-01_1J7,2023-01-01,52,Sunday,6,1J7,7
1,2023-01-01_O2O,2023-01-01,52,Sunday,6,O2O,7
2,2023-01-01_8QR,2023-01-01,52,Sunday,6,8QR,7
3,2023-01-01_L58,2023-01-01,52,Sunday,6,L58,7
4,2023-01-01_UMC,2023-01-01,52,Sunday,6,UMC,7
...,...,...,...,...,...,...,...
78647,2023-05-31_N9K,2023-05-31,22,Wednesday,2,N9K,0
78648,2023-05-31_P6E,2023-05-31,22,Wednesday,2,P6E,0
78649,2023-05-31_BDC,2023-05-31,22,Wednesday,2,BDC,0
78650,2023-05-31_QD6,2023-05-31,22,Wednesday,2,QD6,0


In [27]:
y_test = []

for index, row in tqdm(test_data.iterrows()):
    features = row[nn_model.features_col]
    features = np.array([features.values], dtype=np.float32)
    features = torch.tensor(features, device=device)
    y_pred = nn_model.nns[row['station']](features)
    y_test.append(y_pred.detach().numpy()[0,0])

print("y_test:", y_test)

0it [00:00, ?it/s]

y_test: [130.86179, 34.99848, 41.262234, 18.181189, 116.65656, 455.71344, 185.82552, 691.0861, 412.96127, 611.55896, 246.71904, 511.41132, 498.07693, 684.30054, 639.0829, 94.78643, 464.69223, 586.451, 716.3757, 57.489082, 670.1559, 509.59314, 525.6971, 612.2103, 535.4363, 565.0244, 220.14677, 293.98126, 757.882, 724.9253, 697.31464, 749.6035, 460.0188, 595.4371, 758.8675, 620.42413, 694.30194, 724.41974, 626.0585, 600.06195, 795.70935, 607.2178, 387.6321, 695.56696, 556.69354, 51.292133, 677.463, 127.33702, 284.158, 678.55, 62.998928, 689.5171, 363.97772, 36.195297, 195.7797, 464.08145, 656.824, 61.453053, 67.730576, 722.39185, 28.127895, 45.770557, 144.27487, 25.39041, 595.88007, 601.64813, 145.23834, 135.50162, 521.2439, 775.61975, 234.21745, 573.6239, 682.4651, 87.97221, 568.4422, 639.1058, 514.75903, 729.8649, 590.85205, 631.41986, 691.9562, 558.4594, 787.4146, 593.54865, 638.87024, 643.9839, 692.815, 790.22614, 523.2348, 672.2922, 482.65717, 734.55145, 775.555, 736.96844, 694.9327

In [28]:
y_submit = pd.read_csv('data/submission_example.csv')
y_submit

,index,y
0,2023-01-01_1J7,10000
1,2023-01-01_O2O,10000
2,2023-01-01_8QR,10000
3,2023-01-01_L58,10000
4,2023-01-01_UMC,10000
...,...,...
78647,2023-05-31_N9K,10000
78648,2023-05-31_P6E,10000
78649,2023-05-31_BDC,10000
78650,2023-05-31_QD6,10000


In [29]:
y_submit['y'] = y_test
y_submit

,index,y
0,2023-01-01_1J7,130.861786
1,2023-01-01_O2O,34.998482
2,2023-01-01_8QR,41.262234
3,2023-01-01_L58,18.181189
4,2023-01-01_UMC,116.656563
...,...,...
78647,2023-05-31_N9K,101.008118
78648,2023-05-31_P6E,78.978485
78649,2023-05-31_BDC,65.340790
78650,2023-05-31_QD6,82.957924


In [30]:
filename = 'submissions/MLP_submission_03_17.csv'
y_submit.to_csv(filename, index=False)